In [1]:
!git clone https://github.com/zhangbo2008/hl_detec

Cloning into 'hl_detec'...
remote: Enumerating objects: 3044, done.
remote: Counting objects: 100% (3044/3044), done.
remote: Compressing objects: 100% (2107/2107), done.
remote: Total 3044 (delta 933), reused 3044 (delta 933), pack-reused 0
Receiving objects: 100% (3044/3044), 12.58 MiB | 11.87 MiB/s, done.
Resolving deltas: 100% (933/933), done.


In [2]:
cd hl_detec

/kaggle/working/hl_detec


In [3]:
!rm tvsum50_ver_1_1.tgz -rf  # 先删除旧文件就非常robost
!wget https://huggingface.co/datasets/zhangbo2008/tvsum/resolve/main/tvsum50_ver_1_1.tgz

--2023-06-28 09:36:48--  https://huggingface.co/datasets/zhangbo2008/tvsum/resolve/main/tvsum50_ver_1_1.tgz
Resolving huggingface.co (huggingface.co)... 108.138.94.52, 108.138.94.97, 108.138.94.45, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.52|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cf/2f/cf2f2fea5cd14619dff8af32aded7579b33ded418dc4bb1f84b5099b96a85cd0/407d340bcd06fdc6d17374ebe6760b4a96816bcace228559c8283d9fb2520dea?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tvsum50_ver_1_1.tgz%3B+filename%3D%22tvsum50_ver_1_1.tgz%22%3B&Expires=1688204209&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2NmLzJmL2NmMmYyZmVhNWNkMTQ2MTlkZmY4YWYzMmFkZWQ3NTc5YjMzZGVkNDE4ZGM0YmIxZjg0YjUwOTliOTZhODVjZDAvNDA3ZDM0MGJjZDA2ZmRjNmQxNzM3NGViZTY3NjBiNGE5NjgxNmJjYWNlMjI4NTU5YzgyODNkOWZiMjUyMGRlYT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF

In [4]:
!tar -xvzf tvsum50_ver_1_1.tgz

./WebscopeReadMe.txt
./ydata-tvsum50-v1_1/
./ydata-tvsum50-v1_1/README
./ydata-tvsum50-v1_1/ydata-tvsum50-data.zip
./ydata-tvsum50-v1_1/ydata-tvsum50-matlab.zip
./ydata-tvsum50-v1_1/ydata-tvsum50-thumbnail.zip
./ydata-tvsum50-v1_1/ydata-tvsum50-video.zip


In [5]:
!unzip ./ydata-tvsum50-v1_1/ydata-tvsum50-video.zip

Archive:  ./ydata-tvsum50-v1_1/ydata-tvsum50-video.zip
   creating: video/
  inflating: video/-esJrBWj2d8.mp4   
  inflating: video/0tmA_C6XwfM.mp4   
  inflating: video/37rzWOQsNIw.mp4   
  inflating: video/3eYKfiOEJNs.mp4   
  inflating: video/4wU_LUjG5Ic.mp4   
  inflating: video/91IHQYk1IQM.mp4   
  inflating: video/98MoyGZKHXc.mp4   
  inflating: video/_xMr-HKMfVA.mp4   
  inflating: video/akI8YFjEmUw.mp4   
  inflating: video/AwmHb44_ouw.mp4   
  inflating: video/b626MiF1ew4.mp4   
  inflating: video/Bhxk-O1Y7Ho.mp4   
  inflating: video/byxOvuiIJV0.mp4   
  inflating: video/cjibtmSLxQ4.mp4   
  inflating: video/E11zDS9XGzg.mp4   
  inflating: video/EE-bNr36nyA.mp4   
  inflating: video/eQu1rNs0an0.mp4   
  inflating: video/EYqVtI9YWJA.mp4   
  inflating: video/fWutDQy1nnY.mp4   
  inflating: video/GsAD1KT1xo8.mp4   
  inflating: video/gzDbaEs1Rlg.mp4   
  inflating: video/Hl-__g2gn_A.mp4   
  inflating: video/HT5vyqe0Xaw.mp4   
  inflating: video/i3wAGJaaktw.mp4   
  inflating: 

In [6]:
!pip install av
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 32.3 MB/s eta 0:00:0000:0100:01


In [ ]:
# =======vid path :  /mnt/e/ydata-tvsum50-v1_1/video



import av
import torch
import numpy as np

from transformers import AutoProcessor, AutoModel
from huggingface_hub import hf_hub_download
device = torch.device("cuda" if torch.cuda.is_available()else "cpu") 
np.random.seed(0)
processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

#=============做数据集

import numpy as np

datap='video'
import pandas as pd
anno='ydata-tvsum50-anno.tsv'
info='ydata-tvsum50-info.tsv'


all_video=[]
all_label=[]
chulitvsum=1
idex=0
info = pd.read_csv(info, sep="\t")
if chulitvsum:
 print()


    


print()

#================ferz


print('开始训练')
epoch=10
from torch import nn, optim
# optimizer = optim.Adam(model.parameters(), lr=0.001)
import torch.optim as optim

for p in model.parameters():
    p.requires_grad=False

for p in model.base_model.last.parameters():
    p.requires_grad=True
print()


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
model=model.to(device)
for i in range(epoch):

#   for out3, out4 in zip(all_video,all_label):
#     #  getdata
  for idex in range(len(info)):
    # idex=0
    
    time=[]
    for i in range(len(info)):
        tmp=info.iloc[i].length
        time.append(int(tmp.split(':')[0])*60+int(tmp.split(':')[1]))

    import glob
    files=glob.glob(datap+'/*.mp4')


    print(1)
    dummy=datap+'/'+info.iloc[idex].video_id+'.mp4'  #取第一个做dummytest
    dummy_shipinchang=time[idex] # diyige shipinchnag .
    dummy_fps=1
    import av
    container = av.open(dummy)
    container.streams.video[0].average_rate
    time=container.streams.video[0].duration/10000
    time=dummy_shipinchang
    frames=container.streams.video[0].frames
    fps=frames/time
    print(1)

    indices=[]
    jiange=int(fps*2/8) # 每8个一组
    for i in range(0,frames,jiange):
        indices.append(i)

    print(1) 
    t=[]
    out=[]
    for i in indices:

        t.append(i)
        if len(t)==8:
                out.append(t)
                t=[]
    # if t:
    #     out.append(t)
    hout=out
    print(2)
    #-------每一组的平分.


    # import numpy as np 
    # df = pd.read_csv(anno, sep="\t")  # 用pd速度快.
    # df.iloc[0]
    with open(anno) as f:
        tmp=f.readlines()

    tmp=[''.join(i.strip().split('\t')[2:]).split(',') for i in tmp]
    tmp2=[]
    for i in range(len(tmp)):
        tmp2.append([int(i) for i in tmp[i]])
    tmp=tmp2 
    all=[]
#     for i in tmp:
#         print(len(i))
    for i in range(0,len(tmp),20):
        
        t=(np.array(tmp[i])+np.array(tmp[i+1])+np.array(tmp[i+2]))/3
#         print()
        all.append(t)
    #========改变out平分.
#     print()
    all2=[]
    frames2 = []
    container.seek(0)
    out=sum(out,[])
    for i, frame in enumerate(container.decode(video=0)):
        if i in out:
            frames2.append(frame)
#     print()
    t=[]
    out2=[]
    for i in frames2:

        t.append(i)
        if len(t)==8:
                out2.append(t)
                t=[]
    # if t:
    #     out2.append(t)
#     print(2)
    out3=[]
    for i in out2:

        out3.append(np.stack([x.to_ndarray(format="rgb24") for x in i]))
    print(len(out3),'numberofkeyframe')
    print()
    fenshu=all[idex]
    hout
    out4=[]
    for i in hout:
        ttt=fenshu[i]
        t=np.mean(ttt)/5 #===============guiyihua
        out4.append(t)
    print()
    # all_video.append(out3)
    # all_label.append(out4)













    bs=100
    for i in range(0,len(out3),bs):
        tmp1=out3[i:i+bs]
        tmp2=out4[i:i+bs]



        optimizer.zero_grad()
        
        video=tmp1

        tmp2=torch.tensor(tmp2).to(device)

        # Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor.
        inputs = processor(
            text=[""],
            videos=list(video),
            return_tensors="pt",
            padding=True,
        )
        for i in inputs:
            inputs[i]=inputs[i].to(device)





        #=============收集所有的数据和标签.

    #     print(inputs,66666666666666666)



        # forward pass
        model.train()
        outputs,loss = model(**inputs,return_loss=True,label=tmp2)


        print(loss,'当前损失')
        loss.backward()
        optimizer.step()




print('over_train')
# logits_per_video = outputs.logits_per_video  # this is the video-text similarity score
# probs = logits_per_video.softmax(dim=1)  # we can take the softmax to get the label probabilities
# print(probs)




/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of XCLIPModel were not initialized from the model checkpoint at microsoft/xclip-base-patch32 and are newly initialized: ['last.weight', 'last.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




开始训练

1
1
1
2
189 numberofkeyframe




/kaggle/working/hl_detec/transformers/models/x_clip/modeling_x_clip.py:1597: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).to(torch.float32)
/kaggle/working/hl_detec/transformers/models/x_clip/modeling_x_clip.py:1599: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss=nn.MSELoss()(ans,torch.tensor(label).reshape(-1,1))


tensor(0.3233, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
tensor(0.2677, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
97 numberofkeyframe


tensor(0.3831, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
292 numberofkeyframe


tensor(0.3840, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
tensor(0.2724, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
tensor(0.3032, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
150 numberofkeyframe


tensor(0.2746, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
tensor(0.3254, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
59 numberofkeyframe


tensor(0.1854, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
172 numberofkeyframe


tensor(0.2716, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
tensor(0.4007, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
79 numberofkeyframe


tensor(0.2381, device='cuda:0', grad_fn=<MseLossBackward0>) 当前损失
1
1
1
2
176 numberofkeyframe


tensor

In [ ]:
!fuser -v /dev/nvidia*  #清空显存.
!kill 15821    # 这里面输入号. 号会运行这块之后出现.
!nvidia-smi
torch.cuda.empty_cache()